In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import numpy as np

In [2]:
hop_teaming = pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', 
                          nrows = 100)

In [3]:
hop_teaming.head(10)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880
5,1508252933,1730166109,16,16,78.750,103.169
6,1508076001,1730166109,46,48,72.625,83.263
7,1508085911,1730166125,58,67,23.925,43.923
8,1508093394,1730166125,15,15,37.400,79.602
9,1508132986,1730166125,11,11,97.364,62.152


In [4]:
#Create chunk
chunks = pd.read_csv('Hop_Teaming_Data/DocGraph_Hop_Teaming_2018.csv', chunksize=10000)

In [5]:
next(chunks)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880
...,...,...,...,...,...,...
9995,1508026980,1730477589,36,46,64.239,77.845
9996,1497997050,1730477589,38,43,40.395,60.289
9997,1497977268,1730477761,14,14,37.857,72.533
9998,1508024217,1730477811,12,12,59.000,46.043


**We want to eliminate "accidental" referrals, so filter the hop teaming data so that the transaction_count is at least 50 and the average_day_wait is less than 50.**

In [7]:
chunks = pd.read_csv('Hop_Teaming_Data/DocGraph_Hop_Teaming_2018.csv', chunksize=10000)
accident_filtered = []
for chunk in chunks:
    chunk = chunk[(chunk["transaction_count"] >= 50) & (chunk["average_day_wait"] < 50)]
    accident_filtered.append(chunk)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
len(accident_filtered)

In [8]:
hop = pd.concat(accident_filtered, ignore_index = True)

In [9]:
hop.shape

(34176938, 6)

# nppes data

In [10]:
npi_data = pd.read_csv('data/npidata_pfile_20050523-20230212.csv', nrows = 100)

In [11]:
npi_colnames = list(npi_data.columns)
#npi_colnames

In [12]:
hptc = [i for i in npi_colnames if i.startswith('Healthcare Provider Taxonomy Code_')]
hptc

['Healthcare Provider Taxonomy Code_1',
 'Healthcare Provider Taxonomy Code_2',
 'Healthcare Provider Taxonomy Code_3',
 'Healthcare Provider Taxonomy Code_4',
 'Healthcare Provider Taxonomy Code_5',
 'Healthcare Provider Taxonomy Code_6',
 'Healthcare Provider Taxonomy Code_7',
 'Healthcare Provider Taxonomy Code_8',
 'Healthcare Provider Taxonomy Code_9',
 'Healthcare Provider Taxonomy Code_10',
 'Healthcare Provider Taxonomy Code_11',
 'Healthcare Provider Taxonomy Code_12',
 'Healthcare Provider Taxonomy Code_13',
 'Healthcare Provider Taxonomy Code_14',
 'Healthcare Provider Taxonomy Code_15']

In [13]:
hpptswitch = [i for i in npi_colnames if i.startswith('Healthcare Provider Primary Taxonomy Switch_')]
hpptswitch

['Healthcare Provider Primary Taxonomy Switch_1',
 'Healthcare Provider Primary Taxonomy Switch_2',
 'Healthcare Provider Primary Taxonomy Switch_3',
 'Healthcare Provider Primary Taxonomy Switch_4',
 'Healthcare Provider Primary Taxonomy Switch_5',
 'Healthcare Provider Primary Taxonomy Switch_6',
 'Healthcare Provider Primary Taxonomy Switch_7',
 'Healthcare Provider Primary Taxonomy Switch_8',
 'Healthcare Provider Primary Taxonomy Switch_9',
 'Healthcare Provider Primary Taxonomy Switch_10',
 'Healthcare Provider Primary Taxonomy Switch_11',
 'Healthcare Provider Primary Taxonomy Switch_12',
 'Healthcare Provider Primary Taxonomy Switch_13',
 'Healthcare Provider Primary Taxonomy Switch_14',
 'Healthcare Provider Primary Taxonomy Switch_15']

In [14]:
other_cols = ['NPI', 'Entity Type Code', 'Provider Organization Name (Legal Business Name)',
                                     'Provider Last Name (Legal Name)',
                                     'Provider First Name',
                                     'Provider Middle Name',
                                     'Provider Name Prefix Text',
                                     'Provider Name Suffix Text',
                                     'Provider Credential Text',
                                     'Provider First Line Business Practice Location Address',
                                     'Provider Second Line Business Practice Location Address',
                                     'Provider Business Practice Location Address City Name',
                                     'Provider Business Practice Location Address State Name',
                                     'Provider Business Practice Location Address Postal Code']

In [15]:
ls = hptc + hpptswitch + other_cols
ls

['Healthcare Provider Taxonomy Code_1',
 'Healthcare Provider Taxonomy Code_2',
 'Healthcare Provider Taxonomy Code_3',
 'Healthcare Provider Taxonomy Code_4',
 'Healthcare Provider Taxonomy Code_5',
 'Healthcare Provider Taxonomy Code_6',
 'Healthcare Provider Taxonomy Code_7',
 'Healthcare Provider Taxonomy Code_8',
 'Healthcare Provider Taxonomy Code_9',
 'Healthcare Provider Taxonomy Code_10',
 'Healthcare Provider Taxonomy Code_11',
 'Healthcare Provider Taxonomy Code_12',
 'Healthcare Provider Taxonomy Code_13',
 'Healthcare Provider Taxonomy Code_14',
 'Healthcare Provider Taxonomy Code_15',
 'Healthcare Provider Primary Taxonomy Switch_1',
 'Healthcare Provider Primary Taxonomy Switch_2',
 'Healthcare Provider Primary Taxonomy Switch_3',
 'Healthcare Provider Primary Taxonomy Switch_4',
 'Healthcare Provider Primary Taxonomy Switch_5',
 'Healthcare Provider Primary Taxonomy Switch_6',
 'Healthcare Provider Primary Taxonomy Switch_7',
 'Healthcare Provider Primary Taxonomy Switc

In [16]:
npi_sample = pd.read_csv('nppes_data/npidata_pfile_20050523-20230212.csv', 
                         usecols = ls, nrows= 1000)

In [17]:
npi_sample.shape

(1000, 44)

In [18]:
npi_sample.head(10)

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,...,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15
0,1740284231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1346245800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1487650776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1033113022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1043216138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1104820703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1811996671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1225030232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1194711135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1407848740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
def f(row):
    if row['Healthcare Provider Primary Taxonomy Switch_1'] == 'Y':
        new_code= row['Healthcare Provider Taxonomy Code_1']
        
    elif row['Healthcare Provider Primary Taxonomy Switch_2'] == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_2']
        
    elif 'Healthcare Provider Primary Taxonomy Switch_3' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_3']
    
    elif 'Healthcare Provider Primary Taxonomy Switch_4' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_4']

    elif 'Healthcare Provider Primary Taxonomy Switch_5' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_5']
    
    elif 'Healthcare Provider Primary Taxonomy Switch_6' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_6']
    
    elif 'Healthcare Provider Primary Taxonomy Switch_7' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_7']
    
    elif 'Healthcare Provider Primary Taxonomy Switch_8' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_8']

    elif 'Healthcare Provider Primary Taxonomy Switch_9' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_9']  

    
    elif 'Healthcare Provider Primary Taxonomy Switch_10' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_10']
    
    elif 'Healthcare Provider Primary Taxonomy Switch_11' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_11']
    
    elif 'Healthcare Provider Primary Taxonomy Switch_12' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_12']

    elif 'Healthcare Provider Primary Taxonomy Switch_13' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_13']

    
    elif 'Healthcare Provider Primary Taxonomy Switch_14' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_14']

    elif 'Healthcare Provider Primary Taxonomy Switch_15' == 'Y':
        new_code = row['Healthcare Provider Taxonomy Code_15']
        
    else:
        new_code = row['Healthcare Provider Taxonomy Code_1']
        
    return new_code

In [20]:
npi_sample['code_new'] = npi_sample.apply(f, axis=1)

In [21]:
npi_sample

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,...,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,code_new
0,1740284231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1346245800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1487650776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1033113022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1043216138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1306849922,1.0,NaN,PENRY,DAVID,V,NaN,NaN,APRN,"NP RESOURCES, LLC.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,363LF0000X
996,1124021746,1.0,NaN,COTTER,JOHN,BURLEY,DR.,NaN,MD,1270 PEACH ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207W00000X
997,1851394472,1.0,NaN,PETERS,DAVID,S,NaN,NaN,MD,112 S NORTHWEST HWY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207Q00000X
998,1487657003,1.0,NaN,CULALA,RYAN,MICHAEL,NaN,NaN,PHARM.D.,11521 NE 128TH ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183500000X


In [22]:
cols_to_drop = hpptswitch + hptc

In [23]:
npi_chunks = pd.read_csv('nppes_data/npidata_pfile_20050523-20230212.csv', 
                         chunksize = 10000, 
                         usecols = ls)
x = []
for chunk in npi_chunks:
    chunk['code_new'] = chunk.apply(f, axis = 1)
    chunk.drop(columns=cols_to_drop, inplace = True)
    
    x.append(chunk)

C:\Users\16156\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (71,74,75,78,79,82,83,86,87,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\16156\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (32,67,70,71,74,75,78,79,82,83,86,87,90,91,94) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\16156\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (32,71,74,75,78,79,82,83,86,87,90,91,94) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\16156\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (32,79,82,83,86,87,90,91,94,95,98,99,102) have 

C:\Users\16156\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (32,75,78,79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\16156\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (67,70,71,74,75,78,79,82,83,86) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\16156\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (32,71,74,75,78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\16156\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (32,87,90,91,94,95,98,99,102,103,106) have mixed types.Specify

KeyboardInterrupt: 

In [24]:
import sys
print(sys.getrecursionlimit())

3000


In [52]:
npi_tax_ided = pd.concat(x, ignore_index = True)

In [25]:
#Monica's code

In [26]:
db = sqlite3.connect('data/hop_db.sqlite')

In [27]:
def zip_zeroes(x):
    if len(x) > 5:
        return x.zfill(9)[:5]
    elif len(x) > 0:
        return x.zfill(5)
    else: return np.nan

In [28]:
#table 1: npi data
for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20230212.csv', chunksize = 100000, usecols = ls, low_memory = False)):
    chunk['taxonomy_code'] =  chunk.apply(f, axis = 1)
    chunk = chunk.drop(columns = cols_to_drop)
    chunk['Provider Business Practice Location Address Postal Code'] = chunk['Provider Business Practice Location Address Postal Code'].astype(str).apply(zip_zeroes)
    chunk.columns = [x.lower()
                 .replace('provider ', '')
                 .replace('business ', '')
                 .replace('practice ', '')
                 .replace(' text', '')
                 .replace(' (legal name)', '')
                 .replace(' ', '_') for x in chunk.columns]
    chunk.to_sql('npidata', db, if_exists = 'append', index = False)
    

0it [00:00, ?it/s]

In [29]:
#table 2: hop_teaming data
for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)): 
    chunk = chunk[(chunk["transaction_count"] >= 25) & (chunk["average_day_wait"] < 90)]
    chunk.to_sql('hop', db, if_exists = 'append', index = False)

0it [00:00, ?it/s]

In [30]:
#table 3: taxonomy
tax_deets = pd.read_csv('data/nucc_taxonomy_230.csv')
tax_deets.columns = [x.lower().replace(' ', '_') for x in tax_deets.columns]
tax_deets = tax_deets.rename(columns = {'code' : 'taxonomy_code'})
tax_deets
tax_deets.to_sql('taxonomy', db, if_exists = 'append', index = False)

In [31]:
#table 4: zip_to_cbsa
zip_cbsa = pd.read_excel('data/ZIP_CBSA_122021.xlsx')
zip_cbsa = zip_cbsa.rename(columns = {'usps_zip_pref_city' : 'city',
                              'usps_zip_pref_state' : 'state'})

zip_cbsa['zip'] = zip_cbsa['zip'].astype(str).str.zfill(5)
zip_cbsa.to_sql('zip_to_cbsa', db, if_exists = 'append', index = False)

In [32]:
db.close()

In [33]:
db = sqlite3.connect('data/hop_db.sqlite')

# hop table
db.execute('CREATE INDEX from_npi ON hop(from_npi)')
db.execute('CREATE INDEX to_npi ON hop(to_npi)')
db.execute('CREATE INDEX transaction_count ON hop(transaction_count)')
db.execute('CREATE INDEX average_day_wait ON hop(average_day_wait)')

db.close()

In [36]:
db = sqlite3.connect('data/hop_db.sqlite')

# npidata table
db.execute('CREATE INDEX taxonomy_code ON npidata(taxonomy_code)')
db.execute('CREATE INDEX npi ON npidata(npi)')
db.execute('CREATE INDEX entity_type_code ON npidata(entity_type_code)')
db.execute('CREATE INDEX organization_name ON npidata(organization_name)')
db.execute('CREATE INDEX location_address_city_name ON npidata(location_address_city_name)')
db.execute('CREATE INDEX location_address_state_name ON npidata(location_address_state_name)')
db.execute('CREATE INDEX location_address_postal_code ON npidata(location_address_postal_code)')

db.close()

OperationalError: index taxonomy_code already exists

In [37]:
db = sqlite3.connect('data/hop_db.sqlite')

# taxonomy table
db.execute('CREATE INDEX t_code ON taxonomy(taxonomy_code)')
db.execute('CREATE INDEX display_name ON taxonomy(display_name)')
db.execute('CREATE INDEX classification ON taxonomy(classification)')

db.close()

In [38]:
db = sqlite3.connect('data/hop_db.sqlite')

#zip_cbsa table
db.execute('CREATE INDEX zip ON zip_to_cbsa(zip)')
db.execute('CREATE INDEX city ON zip_to_cbsa(city)')
db.execute('CREATE INDEX state ON zip_to_cbsa(state)')
db.execute('CREATE INDEX cbsa ON zip_to_cbsa(cbsa)')

db.close()